In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras

# Load the models
octrga_model = keras.models.load_model('/content/drive/MyDrive/concatcheck/octrga1.h5', compile=False, custom_objects={'tf': tf})
fundusmsan_model = keras.models.load_model('/content/drive/MyDrive/concatcheck/fundusmsan.h5', compile=False, custom_objects={'tf': tf})

# Set unique names for the input layers
octrga_input = keras.Input(shape=(256, 256, 3), name='octrga_input')  # Adjust the input shape according to your requirements
fundusmsan_input = keras.Input(shape=(256, 256, 3), name='fundusmsan_input')  # Adjust the input shape according to your requirements

# Connect the input layers to their respective models
octrga_output = octrga_model(octrga_input)
fundusmsan_output = fundusmsan_model(fundusmsan_input)

# Concatenate the outputs
concatenated = keras.layers.Concatenate()([octrga_output, fundusmsan_output])

# Add a fully connected layer for prediction
fc_layer = keras.layers.Dense(64, activation='relu')(concatenated)
predictions = keras.layers.Dense(3, activation='softmax')(fc_layer)  # Replace 'num_classes' with the number of classes

# Create a new combined model with unique names
combined_model = keras.models.Model(inputs=[octrga_input, fundusmsan_input], outputs=predictions, name='combined_model')

# Compile the combined model
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Split the data for the octrga model
oct_train_data, oct_test_data, oct_train_labels, oct_test_labels = train_test_split(oct_data, oct_labels, test_size=0.2, random_state=42)

# Split the data for the fundusmsan model
fundus_train_data, fundus_test_data, fundus_train_labels, fundus_test_labels = train_test_split(fundus_data, fundus_labels, test_size=0.2, random_state=42)

# Train the combined model
combined_model.fit([oct_train_data, fundus_train_data], train_labels, epochs=10, batch_size=32)

# Evaluate the combined model
test_loss, test_accuracy = combined_model.evaluate([oct_test_data, fundus_test_data], test_labels)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


OSError: ignored